<a href="https://colab.research.google.com/github/szheng3/recommendation-system/blob/main/Explore_CQL/DLR2/HM_CQL_SA2C_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SA2C-SASRec Recommender on H&M Data with Conservative Q-learning

In this notebook we train an SA2C_SASrec model without and with CQL to train for recommendation on H&M dataset.

1. Clone the git repository containing all the source code

In [3]:
# cloning the git repository
!git clone https://github.com/szheng3/recommendation-system

Cloning into 'recommendation-system'...
remote: Enumerating objects: 1622, done.
remote: Counting objects: 100% (403/403), done.
remote: Compressing objects: 100% (216/216), done.
remote: Total 1622 (delta 226), reused 331 (delta 186), pack-reused 1219
Receiving objects: 100% (1622/1622), 11.71 MiB | 9.93 MiB/s, done.
Resolving deltas: 100% (856/856), done.


2. Install the missing libraries (specifically for google colab)

In [4]:
# installing missing libraries (specifically for google colab)
!pip install trfl d3rlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.6/317.6 kB 37.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.8 MB/s eta 0:00:00
  Created wheel for d3rlpy: filename=d3rlpy-1.1.1-cp310-cp310-linux_x86_64.whl size=1257776 sha256=54852be99a426baac3ae8bd79c6767e7cd7c28089d96301f8a9d9c8c3413fc6d
  Stored in directory: /root/.cache/pip/wheels/fb/44/c4/aab790777bd0d1358b9e8cc391621a4e0e2b10e7cd03406ad6
Successfully built d3rlpy


3. Download the H&M Dataset transaction log file from AWS S3 bucket

In [5]:
# downloading the "transactions_train.csv" datafile from the H&M Dataset
!wget https://aipi590.s3.amazonaws.com/transactions_train.csv -P "/content/recommendation-system/Explore_CQL/Data/HM_data/"

--2023-05-03 16:29:52--  https://aipi590.s3.amazonaws.com/transactions_train.csv
Resolving aipi590.s3.amazonaws.com (aipi590.s3.amazonaws.com)... 52.216.237.43, 52.217.47.52, 3.5.3.139, ...
Connecting to aipi590.s3.amazonaws.com (aipi590.s3.amazonaws.com)|52.216.237.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3488002253 (3.2G) [text/csv]
Saving to: ‘/content/recommendation-system/Explore_CQL/Data/HM_data/transactions_train.csv’

transactions_train. 100%[===================>]   3.25G  12.7MB/s    in 4m 30s  

2023-05-03 16:34:23 (12.3 MB/s) - ‘/content/recommendation-system/Explore_CQL/Data/HM_data/transactions_train.csv’ saved [3488002253/3488002253]



4. Run the "gen_replay_buffer.py" script to pre-process data and generate replay buffer

In [12]:
# Executing the python script "gen_replay_buffer" to sample the given H&M dataset
!python "/content/recommendation-system/Explore_CQL/DLR2/src/gen_replay_buffer_HM.py" --size=100000 --data="/content/recommendation-system/Explore_CQL/Data/HM_data"


Start reading all transaction data ...
Finish reading in 00:00:26

Filter and save all valid sampled data
/content/recommendation-system/Explore_CQL/DLR2/src/gen_replay_buffer_HM.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_sessions['valid_session'] = sampled_sessions.session_id.map(sampled_sessions.groupby('session_id')['item_id'].size() > 2)
Index(['timestamp', 'session_id', 'item_id', 'price', 'sales_channel_id'], dtype='object')

Start counting popularity ...
1049917it [00:35, 29293.42it/s]
Popularity finished in 00:00:35

Start spliting into train, val, test data ...

           Generate Replay Buffer:
                Total Session Size : 100000
                     Train:      70000 ids | 737354 actions
                   

5. Run script to begin training and evaluate the SASRec-SNQN model.

In [13]:
# Executing the python script "SA2C_v3.py" to train a SA2C-SASrec model on the sampled data and evaluate it on validation data using HR (hit rate) and NDCG metrics for different values of k
!python "/content/recommendation-system/Explore_CQL/DLR2/src/SA2C_v3_5.py" --model=SASRec --data="/content/recommendation-system/Explore_CQL/Data/HM_data/" --epoch=20

2023-05-03 22:42:04.817340: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 22:42:05.804693: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Not using CQL loss
2023-05-03 22:42:07.696720: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype bool
	 [[{{node Placeholder}}]]
2023-05-03 22:42:07.729779: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning 

In [14]:
# Executing the python script "SA2C_v3.py" to train a SA2C-SASrec model on the sampled data with CQL and evaluate it on validation data using HR (hit rate) and NDCG metrics for different values of k
!python "/content/recommendation-system/Explore_CQL/DLR2/src/SA2C_v3_5.py" --model=SASRec --CQL_alpha=1.0 --data="/content/recommendation-system/Explore_CQL/Data/HM_data/" --epoch=20

2023-05-04 05:30:34.409139: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 05:30:35.407704: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using CQL loss.
2023-05-04 05:30:37.244359: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype bool
	 [[{{node Placeholder}}]]
2023-05-04 05:30:37.276582: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUM